# Preparing datasets: Apr'24
This notebook is meant to generate a new dataset (only for April 2024) based on *Citibike Trip data* that can be used to determine **bike rentals** based on a <u>fixed time resolution</u>. 

## Preparing Tripdata

In [1]:
import pandas as pd
citi_apr_24 = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Datasets/Citibike Trip Data/202404-citibike-tripdata.csv")
citi_apr_24.head()

C:\Users\singh\AppData\Local\Temp\ipykernel_11404\3679919481.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citi_apr_24 = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Datasets/Citibike Trip Data/202404-citibike-tripdata.csv")


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F561526822C9D60B,electric_bike,2024-04-27 13:56:13.940,2024-04-27 14:05:23.629,FDR Drive & E 35 St,6230.04,E 10 St & 2 Ave,5746.02,40.743955,-73.971391,40.729708,-73.986598,member
1,359BAF91507F4998,electric_bike,2024-04-25 15:23:14.529,2024-04-25 15:27:52.895,Forsyth St & Grand St,5382.07,E 10 St & 2 Ave,5746.02,40.717741,-73.993388,40.729708,-73.986598,member
2,AAEE95A1C0106C97,electric_bike,2024-04-06 11:15:18.132,2024-04-06 11:22:10.081,E 20 St & 2 Ave,5971.08,Mott St & Prince St,5561.04,40.735790,-73.981693,40.723180,-73.994800,member
3,95B077C9C619D404,electric_bike,2024-04-06 16:19:25.749,2024-04-06 16:21:43.098,Eastern Pkwy & Washington Ave,3928.08,Eastern Pkwy & Franklin Ave (SW Corner),3919.12,40.671649,-73.963115,40.670529,-73.958222,member
4,1A33C864454C4692,electric_bike,2024-04-10 17:40:14.700,2024-04-10 17:48:11.571,W 27 St & 6 Ave,6215.07,E 25 St & 1 Ave,6004.07,40.745446,-73.990591,40.738177,-73.977387,member


In [2]:
# The dataset needs timeline trimming - exclude Mar data
print(min(citi_apr_24["started_at"]), max(citi_apr_24["started_at"]))

2024-03-31 03:37:14.624 2024-04-30 23:57:38.389


In [3]:
# There are NaN values in the dataset
sum(citi_apr_24["start_station_name"].isna())

2506

In [5]:
# If the dataset is filtered for 'classic bikes' all starting stations should be visible
sum(citi_apr_24[citi_apr_24["rideable_type"] == "classic_bike"]["start_station_name"].isna())

0

In [8]:
# filtering the dataset
cbikes = citi_apr_24[citi_apr_24["rideable_type"] == "classic_bike"]
cbikes["rideable_type"].unique()

array(['classic_bike'], dtype=object)

In [9]:
# reducing columns
cbikes = cbikes[["ride_id", "started_at", "ended_at", "start_station_name", "start_station_id", "end_station_name", "end_station_id", "start_lat", "start_lng", "end_lat", "end_lng"]]

# ride_id can be used to count trips
print(len(cbikes), len(cbikes["ride_id"].unique()))

1109266 1109266


Instead of relying on station_ids provided by **Citibike**, the manually constructed IDs can be entered in dataset. This will make it easy for referencing and connecting *rentals* table below with this original filtered dataset.<br><br>
<u>Note</u>: Original filtered dataset means data for classic bikes where NaN values and redundant columns are removed i.e. **cbikes** dataframe.

## Creating Rentals Template
The new template should contain stations with their location and time information included. Each row represents <u> one unit of chosen time resolution</u>.
<br><br>
**NOTE:** Station ID needs to be retained because there are some stations with the same name but slightly different locations!
<br><br>
In the dataset, some IDs are stored as text and others are stored as numbers. Converting th data type to *string* does not work because **\n** and some other text is being added automatically.
<br><br>
**Solution:**<br>
The stations can be manually given unique ids for the purpose of indentification.

In [10]:
# Creating the new dataset
rentals = pd.DataFrame()
rentals[["name", "lat", "lng"]] = cbikes[["start_station_name", "start_lat", "start_lng"]].drop_duplicates()
rentals = rentals.sort_values(by = ["name"], ignore_index=True)

# Adding IDs manually
rentals["station_id"] = range(len(rentals))

# Verifying the number of stations
print(len(cbikes[["start_station_name", "start_lat", "start_lng"]].drop_duplicates()), len(rentals["station_id"]))

2125 2125


We have one additional station in this dataset.
## Reformatting Tripdata
Making some changes to the trip data before exporting.

In [11]:
# Copying cbikes df
cbikes_altered = cbikes.copy()

# Merging the two dfs
new_df = pd.merge(
    left=cbikes_altered, 
    right=rentals,
    how='left',
    left_on=['start_station_name', 'start_lat', 'start_lng'],
    right_on=['name', 'lat', 'lng'],
)

# Preserving bike trip info only with the starting station info
cbikes_altered = new_df[["ride_id", "started_at", "start_station_name", "station_id", "start_lat", "start_lng"]]

# Checking validity
print(len(cbikes_altered["station_id"].unique()))
cbikes_altered.head()

2125


,ride_id,started_at,start_station_name,station_id,start_lat,start_lng
0,DD0132B9CD6B63F7,2024-04-10 20:40:55.697,Cleveland Pl & Spring St,776,40.722104,-73.997249
1,612C8DB5E41ADD01,2024-04-12 07:26:22.776,E 20 St & 2 Ave,1000,40.735877,-73.982050
2,61191D9EEA6C7AD7,2024-04-22 18:51:02.476,Dock 72 Way & Market St,879,40.699850,-73.971410
3,4D2C34803BDAC042,2024-04-13 22:54:40.756,E 12 St & 3 Ave,917,40.732233,-73.988900
4,E02B369B26426652,2024-04-13 22:54:47.285,E 12 St & 3 Ave,917,40.732233,-73.988900


In [16]:
# exporting to 'C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 5 (TU Dresden)\Research Task - Spatial Modelling\Code'
cbikes_altered.to_csv("Tripdata_with_manual_ids_apr.csv", index=False)

## Defining time resolution
Here time units are defined that will later be merged with the *rentals* template.

In [12]:
# defining the 7 time units for a given date
time_res = ['08:00:00.000', '10:00:00.000', '12:00:00.000', '14:00:00.000', '16:00:00.000', '18:00:00.000', '20:00:00.000']

# repeating time units 30 times - till 30 Apr
time_res = time_res*30

# creating dates
dates_included = []
while len(dates_included) < 30:
    dates_included.append('2024-04-01')

# days list
nums = ['01', '02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29', '30']

# correcting days
for i in range(len(nums)):
    dates_included[i] = dates_included[i][:-2] + nums[i]

dates_included[:5]

['2024-04-01', '2024-04-02', '2024-04-03', '2024-04-04', '2024-04-05']

Now, we are supposed to combine the date and the time information together. Each day has 7 time units, so each day must be repeated 7 times to accodomate the time resolution.

In [13]:
# repeating each element 7 times
dates_included = [element for element in dates_included for _ in range(7)]

# Adding time dimension
date_time = dates_included.copy()

for t in range(0,len(dates_included)):
    date_time[t] = dates_included[t] + ' ' + time_res[t]

date_time[:7]

['2024-04-01 08:00:00.000',
 '2024-04-01 10:00:00.000',
 '2024-04-01 12:00:00.000',
 '2024-04-01 14:00:00.000',
 '2024-04-01 16:00:00.000',
 '2024-04-01 18:00:00.000',
 '2024-04-01 20:00:00.000']

## Adding time dimensionality to the *rentals* template
A time resolution needs to be added so that number of rentals can be calculated in terms of that time unit, per station.

In [14]:
# rows needed for each station
len(date_time)

210

In [15]:
# duplicating each station 210 times
rentals = rentals.loc[rentals.index.repeat(210)].reset_index(drop=True)

# Repeating date_time: 2125 unique stations
date_time = date_time*2125
rentals["datetime"] = date_time
rentals.head()

,name,lat,lng,station_id,datetime
0,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 08:00:00.000
1,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 10:00:00.000
2,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 12:00:00.000
3,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 14:00:00.000
4,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 16:00:00.000


## Calculating Demand
The rentals are calculated for every time unit, per day per station. The dataset is being prepared only until April 30, 2024.

In [17]:
# importing
tripdata = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Code/Tripdata_with_manual_ids_apr.csv")

# adding a rentals column 
rentals['#_rentals'] = 0

rentals.head()

,name,lat,lng,station_id,datetime,#_rentals
0,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 08:00:00.000,0
1,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 10:00:00.000,0
2,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 12:00:00.000,0
3,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 14:00:00.000,0
4,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 16:00:00.000,0


In [18]:
# Updating the rentals dataset
rentals['year'] = pd.Series([x.date().year for x in pd.to_datetime(rentals["datetime"])])
rentals['month'] = pd.Series([x.date().month for x in pd.to_datetime(rentals["datetime"])])
rentals['day'] = pd.Series([x.date().day for x in pd.to_datetime(rentals["datetime"])])

rentals['hour'] = pd.Series([x.time().hour for x in pd.to_datetime(rentals["datetime"])])
rentals['minute'] = pd.Series([x.time().minute for x in pd.to_datetime(rentals["datetime"])])
rentals['second'] = pd.Series([x.time().second for x in pd.to_datetime(rentals["datetime"])])
rentals['microsecond'] = pd.Series([x.time().microsecond for x in pd.to_datetime(rentals["datetime"])])

# Updating the trips dataset
tripdata['started_at_year'] = pd.Series([x.date().year for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_month'] = pd.Series([x.date().month for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_day'] = pd.Series([x.date().day for x in pd.to_datetime(tripdata["started_at"])])

tripdata['started_at_hour'] = pd.Series([x.time().hour for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_minute'] = pd.Series([x.time().minute for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_second'] = pd.Series([x.time().second for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_microsecond'] = pd.Series([x.time().microsecond for x in pd.to_datetime(tripdata["started_at"])])

In [19]:
# The tripdata has some misleading March data that should be removed
min(tripdata["started_at_month"])

3

In [20]:
# After Correction
tripdata = tripdata[tripdata["started_at_month"] > 3]
min(tripdata["started_at_month"])

4

In [21]:
# Calculating demand from tripdata for Apr 2024
for k in range(1, 31):
    
    # determining demand for 2024; 08-10hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 8) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] < 10) & (tripdata["started_at_hour"] >= 8) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 10-12hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 10) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 10) & (tripdata["started_at_hour"] < 12) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 12-14hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 12) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 12) & (tripdata["started_at_hour"] < 14) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 14-16hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 14) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 14) & (tripdata["started_at_hour"] < 16) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 16-18hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 16) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 16) & (tripdata["started_at_hour"] < 18) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 18-20hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 18) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 18) & (tripdata["started_at_hour"] < 20) & (tripdata["station_id"] == i)])
    
    # determining demand for 2024; 20-22hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 20) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 20) & (tripdata["started_at_hour"] < 22) & (tripdata["station_id"] == i)])

In [22]:
# total trips on apr 1 2024
print(len(tripdata[(tripdata["started_at_year"] == 2024)&(tripdata["started_at_day"] == 1)&(tripdata["started_at_hour"] >= 8)&(tripdata["started_at_hour"] < 22)]))

# rentals recorded for apr 1 2024 
sum(rentals.loc[(rentals["year"] == 2024) & (rentals["day"] == 1),"#_rentals"])

26351


26351

Hence, the dataset is correct!

In [23]:
# Created dataset
rentals_final = rentals.copy()

# removing unneeded columns
rentals_final.drop(columns=['minute','second','microsecond'], inplace=True)

# final look
rentals_final.head()

,name,lat,lng,station_id,datetime,#_rentals,year,month,day,hour
0,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 08:00:00.000,3,2024,4,1,8
1,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 10:00:00.000,0,2024,4,1,10
2,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 12:00:00.000,0,2024,4,1,12
3,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 14:00:00.000,1,2024,4,1,14
4,1 Ave & E 110 St,40.792327,-73.9383,0,2024-04-01 16:00:00.000,3,2024,4,1,16


In [24]:
# exporting
rentals_final.to_csv("rentals_with_demand_new_time_units_apr.csv", index=False)